In [ ]:
import pandas as pd
import wrds
from datetime import date, timedelta
import re
from dateutil.relativedelta import relativedelta
import numpy as np
import math
from numba import njit
from dataclasses import dataclass
import os
#google colab libraries
# import os
# from google.colab import drive
# drive.mount('/content/drive')



In [ ]:
ff = pd.read_parquet("/content/drive/MyDrive/Dissertation/data/ff_factors_2005_2022.parquet") #ken french risk free is available on github
db =wrds.Connection()

In [ ]:
TICKERS_FILE = "/content/drive/MyDrive/Dissertation/sp500_tickers_all_2000_2023.txt"

#import options
START, END = pd.Timestamp("2005-01-01"), pd.Timestamp("2022-12-31")
CHUNK = 1000
USE_ATM_SELECTION = True
MIN_PRICE = 5.0
OP_LIB  = "optionm"         
ALL_LIB = "optionm_all"

MIN_ST_PRICE = 5.0

def third_friday(y, m):
    d = date(y, m, 15)
    while d.weekday() != 4:  # Friday
        d += timedelta(days=1)
    return d

def next_bday(d):
    d = d + timedelta(days=1)
    while d.weekday() >= 5:  # skip Sat/Sun
        d += timedelta(days=1)
    return d

def table_cols(lib, tbl):
    desc = db.describe_table(lib, tbl)
    col_field = 'name' if 'name' in desc.columns else desc.columns[0]
    return [str(x).lower() for x in desc[col_field].tolist()]

def pick(cols, candidates):
    for c in candidates:
        if c in cols: return c

    for c in candidates:
        for col in cols:
            if c in col: return col
    return None

def chunks(lst, n=800):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

tickers = [t.strip().upper() for t in open(TICKERS_FILE) if t.strip()]
tickers = sorted(set(tickers))

#map the tickers with the secid
def find_symbol_map(tickers):
    cands = []
    for t in db.list_tables(OP_LIB):
        tl = t.lower()
        if tl in {"security","secnmd"} or ("sec" in tl and any(x in tl for x in ["name","nmd","nme"])):
            cands.append(t)
    if not cands:
        raise SystemExit("No mapping table found (optionm.security / optionm.secnmd).")

    out_rows = []
    for tbl in cands:
        cols = table_cols(OP_LIB, tbl)
        id_col   = pick(cols, ["secid","securityid","sec_id"])
        sym_col  = pick(cols, ["symbol","ticker","sec_symbol","issue_symbol","root","sym_root"])
        from_col = pick(cols, ["effdt","startdate","start_date","from","fromdate","startdt","effective_from"])
        to_col   = pick(cols, ["enddt","end_date","thru","thrudate","effective_to","to","enddt"])
        if (id_col is None) or (sym_col is None):
            continue

        pull_cols = [id_col, sym_col] + ([from_col] if from_col else []) + ([to_col] if to_col else [])
        found = []
        for tk_chunk in chunks(tickers, 300):
            syms = ",".join(["'%s'" % s.replace("'", "''") for s in tk_chunk])
            sql = f"SELECT {', '.join(pull_cols)} FROM {OP_LIB}.{tbl} WHERE upper({sym_col}) IN ({syms})"
            df = db.raw_sql(sql)
            if not df.empty:
                df.columns = [c.lower() for c in df.columns]
                df.rename(columns={id_col.lower():"secid", sym_col.lower():"ticker"}, inplace=True)
                df["ticker"] = df["ticker"].astype(str).str.upper()
                if from_col: df.rename(columns={from_col.lower():"start_eff"}, inplace=True)
                if to_col:   df.rename(columns={to_col.lower():"end_eff"},   inplace=True)
                keep_cols = ["secid","ticker"] + (["start_eff"] if from_col else []) + (["end_eff"] if to_col else [])
                found.append(df[keep_cols])

        if found:
            m = pd.concat(found, ignore_index=True).drop_duplicates()
            out_rows.append(m)

    if not out_rows:
        raise SystemExit("Could not map any tickers to secid (check your tickers or OptionMetrics tables).")

    m = pd.concat(out_rows, ignore_index=True).drop_duplicates()

    if {"start_eff","end_eff"}.issubset(m.columns):
        for c in ["start_eff","end_eff"]:
            m[c] = pd.to_datetime(m[c], errors="coerce")
        mask = (
            (m["start_eff"].isna() | (m["start_eff"] <= END)) &
            (m["end_eff"].isna()   | (m["end_eff"] >= START))
        )
        m = m.loc[mask]

    m = m[["secid","ticker"]].dropna().drop_duplicates()
    m["secid"] = m["secid"].astype(int)
    return m

sec_map = find_symbol_map(tickers)
secids = sorted(sec_map["secid"].unique())
print(f"[INFO] Mapped {len(secids)} secids for {sec_map['ticker'].nunique()} tickers.")
if len(secids) == 0:
    raise SystemExit("Mapping returned 0 secids. Stop.")

#monthly schedule
months = pd.period_range(START, END, freq="M")
schedule = pd.DataFrame({
    "month": months.to_timestamp(how="start"),
    "formation_date": [pd.Timestamp(next_bday(third_friday(p.year,p.month))) for p in months],
    "next_exp_friday": [pd.Timestamp(third_friday((p+1).year,(p+1).month)) for p in months],
})
schedule["next_exp_saturday"] = schedule["next_exp_friday"] + pd.Timedelta(days=1)

op_tbls = set(t.lower() for t in db.list_tables(OP_LIB))
OP_BY_YEAR = {}
for t in op_tbls:
    m = re.fullmatch(r"opprcd(\d{4})", t)
    if m:
        OP_BY_YEAR[int(m.group(1))] = t
if not OP_BY_YEAR:
    raise SystemExit("No opprcdYYYY tables in optionm.*")

probe_year = max(min(OP_BY_YEAR), min(max(OP_BY_YEAR), START.year))
op_cols = table_cols(OP_LIB, OP_BY_YEAR[probe_year])
OP_STRIKE = pick(op_cols, ["strike_price","strike"])
OP_BID    = pick(op_cols, ["best_bid","bid"])
OP_ASK    = pick(op_cols, ["best_offer","ask"])
OP_IV     = pick(op_cols, ["impl_volatility","implied_volatility","iv"])
for name,val in [("OP_STRIKE",OP_STRIKE),("OP_BID",OP_BID),("OP_ASK",OP_ASK),("OP_IV",OP_IV)]:
    if val is None:
        raise SystemExit(f"Missing column in opprcdYYYY: {name}")
    
if USE_ATM_SELECTION:
    all_tbls = set(t.lower() for t in db.list_tables(ALL_LIB))
    SECPRD_UNIFIED = "secprd" in all_tbls
    sc_cols = table_cols(ALL_LIB,"secprd") if SECPRD_UNIFIED else table_cols(OP_LIB, f"secprd{probe_year}")
    SC_CLOSE = pick(sc_cols, ["close","close_price","closing_price","prc","price"])
    if SC_CLOSE is None:
        raise SystemExit(f"Missing close price column in secprd: {sc_cols}")


#extract the data 
rows, month_stats = [], []
for _, r in schedule.iterrows():
    form_dt = r['formation_date'].date()
    exp_fri = r['next_exp_friday'].date()
    exp_sat = r['next_exp_saturday'].date()
    exdate_list = [exp_fri, exp_sat]

    
    probe_dt = form_dt
    pulled = None
    for _ in range(5):
        y = pd.Timestamp(probe_dt).year
        op_tbl = OP_BY_YEAR.get(y)
        if op_tbl:
            parts = []
            for sec_chunk in chunks(secids, CHUNK):
                sql = f"""
                  SELECT secid, date, exdate, cp_flag,
                         {OP_STRIKE} AS k,
                         {OP_BID}    AS bid,
                         {OP_ASK}    AS ask,
                         volume, open_interest,
                         {OP_IV}     AS iv,
                         delta, gamma, vega, theta
                  FROM {OP_LIB}.{op_tbl}
                  WHERE cp_flag = 'C'
                    AND date    = '{probe_dt}'
                    AND exdate IN ('{exdate_list[0]}','{exdate_list[1]}')
                    AND secid IN ({",".join(map(str,sec_chunk))})
                """
                df = db.raw_sql(sql)
                if not df.empty:
                    df.columns = [c.lower() for c in df.columns]
                    parts.append(df)
            if parts:
                pulled = pd.concat(parts, ignore_index=True)
                pulled['date']   = pd.to_datetime(pulled['date'],   errors='coerce').dt.normalize()
                pulled['exdate'] = pd.to_datetime(pulled['exdate'], errors='coerce').dt.normalize()
                pulled['secid']  = pulled['secid'].astype(int)
                break
        probe_dt = (pd.Timestamp(probe_dt) + pd.Timedelta(days=1)).date()

    if pulled is None or pulled.empty:
        month_stats.append((form_dt, 0, 0))
        continue

    pulled = pulled.merge(sec_map, on="secid", how="left")
    pre = len(pulled)   


    if USE_ATM_SELECTION:
        pulled['moneyness'] = pulled['s_t'] / pulled['k']
        pulled = pulled[(pulled['moneyness']>=0.8) & (pulled['moneyness']<=1.2)]
        if pulled.empty:
            month_stats.append((probe_dt, pre, 0)); continue
        pulled['dist'] = (pulled['moneyness'] - 1.0).abs()
        pulled['is_otm_pref'] = (pulled['moneyness'] <= 1.0).astype(int)
        pulled = (pulled.sort_values(['secid','dist','is_otm_pref'], ascending=[True,True,False])
                        .groupby('secid', as_index=False).head(1))

    pulled['formation_date'] = pd.to_datetime(pd.Timestamp(probe_dt).normalize())
    pulled['next_expiry']    = pd.to_datetime(pulled['exdate'], errors='coerce').dt.normalize()
    pulled['days_to_expiry'] = (pulled['next_expiry'] - pulled['formation_date']).dt.days

    month_stats.append((probe_dt, pre, len(pulled)))
    rows.append(pulled)

out = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

In [ ]:
SP500_CSV = "/content/drive/MyDrive/Dissertation/sp500_monthly_tickers_2005_2022.csv" #monthly tickers on sp500 available on github
sp = pd.read_csv(SP500_CSV)
sp.columns = sp.columns.str.strip().str.lower()
#formating the list
sp['month'] = pd.to_datetime(sp['date'], errors='coerce').dt.to_period('M').dt.to_timestamp('D').dt.normalize()
def _explode_tickers(cell: str):
    s = str(cell)
    s = re.sub(r'[\[\]\{\}"\']', '', s)     
    s = re.sub(r'[,\|;]+', ' ', s)          
    s = re.sub(r'\s+', ' ', s).strip()     
    if not s:
        return []
    toks = s.split(' ')

    toks = [re.sub(r'[.\-]', '', t.upper()) for t in toks if t]
    return toks

sp['ticker'] = sp['tickers'].dropna().apply(_explode_tickers)
sp = sp.explode('ticker')
sp = sp[['month','ticker']].dropna().drop_duplicates()

if 'formation_date' in out.columns:
    form_col = 'formation_date'
elif 'date' in out.columns:
    form_col = 'date'
else:
    raise ValueError("Δεν βρίσκω ούτε 'formation_date' ούτε 'date' στο out.")

out = out.copy()
out[form_col] = pd.to_datetime(out[form_col], errors='coerce')
out['month']  = out[form_col].dt.to_period('M').dt.to_timestamp('D').dt.normalize()

if 'ticker' not in out.columns:
    raise ValueError("Το out δεν έχει στήλη 'ticker'. Δες το helper στο τέλος για να το προσθέσεις.")

out['ticker_norm'] = out['ticker'].astype(str).str.upper().str.replace(r'[.\-]', '', regex=True)

#filter and keep what matches
sp['in_sp500'] = True
keep = out.merge(sp.rename(columns={'ticker':'ticker_norm'}),
                 on=['month','ticker_norm'], how='left')

out_sp = keep.loc[keep['in_sp500'].fillna(False)].drop(columns=['in_sp500'])
out_sp = out_sp.drop(columns=['ticker_norm'])



In [ ]:
OPTS = out_sp.copy()                   
OP_LIB, ALL_LIB = "optionm", "optionm_all"
S0_BACKUP_DAYS = 4                      
ST_WINDOW_DAYS = 7                     

def table_cols(lib, tbl):
    desc = db.describe_table(lib, tbl)
    col_field = 'name' if 'name' in desc.columns else desc.columns[0]
    return [str(x).lower() for x in desc[col_field].tolist()]

all_tbls = set(t.lower() for t in db.list_tables(ALL_LIB))
SECPRD_UNIFIED = "secprd" in all_tbls

#find the correct name of the column fot the closing price
probe_year = OPTS['formation_date'].min().year if len(OPTS) else 2005
sc_cols = table_cols(ALL_LIB,"secprd") if SECPRD_UNIFIED else table_cols(OP_LIB, f"secprd{probe_year}")
SC_CLOSE = next((c for c in ["close","close_price","closing_price","prc","price"] if c in sc_cols), None)
if SC_CLOSE is None:
    raise SystemExit(f"Δεν βρέθηκε στήλη τιμής στο secprd. Columns: {sc_cols}")

OPTS['formation_date'] = pd.to_datetime(OPTS['formation_date']).dt.normalize()
OPTS['next_expiry']    = pd.to_datetime(OPTS['next_expiry']).dt.normalize()
OPTS['secid']          = OPTS['secid'].astype(int)

#S0 on expiration
def fetch_S0_for_date(d, secids):
    secids_sql = ",".join(map(str, secids))
    if SECPRD_UNIFIED:
        sql = f"""
          SELECT secid, date, {SC_CLOSE} AS close
          FROM {ALL_LIB}.secprd
          WHERE secid IN ({secids_sql}) AND date BETWEEN '{(d - pd.Timedelta(days=S0_BACKUP_DAYS)).date()}'
                                               AND '{d.date()}'
        """
    else:
        y0, y1 = (d - pd.Timedelta(days=S0_BACKUP_DAYS)).year, d.year
        parts = []
        for y in sorted({y0,y1}):
            sql = f"""
              SELECT secid, date, {SC_CLOSE} AS close
              FROM {OP_LIB}.secprd{y}
              WHERE secid IN ({secids_sql}) AND date BETWEEN '{(d - pd.Timedelta(days=S0_BACKUP_DAYS)).date()}'
                                                   AND '{d.date()}'
            """
            parts.append(db.raw_sql(sql))
        df = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()
        df.columns = [c.lower() for c in df.columns]
        if not df.empty:
            df['date'] = pd.to_datetime(df['date']).dt.normalize()
        return (df.sort_values(['secid','date'])
                  .groupby('secid', as_index=False).tail(1)  # max date ≤ d
                  .rename(columns={'date':'s0_date','close':'S0'}))

    df = db.raw_sql(sql)
    df.columns = [c.lower() for c in df.columns]
    if df.empty:
        return pd.DataFrame(columns=['secid','s0_date','S0'])
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    return (df.sort_values(['secid','date'])
              .groupby('secid', as_index=False).tail(1)
              .rename(columns={'date':'s0_date','close':'S0'}))

#ST on the expiration day
def fetch_ST_for_exdate(exd, secids):
    secids_sql = ",".join(map(str, secids))
    start = (exd - pd.Timedelta(days=ST_WINDOW_DAYS)).date()
    stop  = exd.date()
    if SECPRD_UNIFIED:
        sql = f"""
          SELECT secid, date, {SC_CLOSE} AS close
          FROM {ALL_LIB}.secprd
          WHERE secid IN ({secids_sql}) AND date BETWEEN '{start}' AND '{stop}'
        """
        df = db.raw_sql(sql)
    else:
        y0, y1 = (exd - pd.Timedelta(days=ST_WINDOW_DAYS)).year, exd.year
        parts = []
        for y in sorted({y0,y1}):
            sql = f"""
              SELECT secid, date, {SC_CLOSE} AS close
              FROM {OP_LIB}.secprd{y}
              WHERE secid IN ({secids_sql}) AND date BETWEEN '{start}' AND '{stop}'
            """
            parts.append(db.raw_sql(sql))
        df = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()

    df.columns = [c.lower() for c in df.columns]
    if df.empty:
        return pd.DataFrame(columns=['secid','st_date','ST'])
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    return (df.sort_values(['secid','date'])
              .groupby('secid', as_index=False).tail(1)   # max date ≤ exdate
              .rename(columns={'date':'st_date','close':'ST'}))

OPTS = OPTS.sort_values('formation_date')
s0_rows, st_rows = [], []

for d, dfm in OPTS.groupby('formation_date'):
    secids = sorted(dfm['secid'].unique().tolist())
    if not secids:
        continue
    s0 = fetch_S0_for_date(d, secids)
    s0['formation_date'] = d
    s0_rows.append(s0)

for exd, dfe in OPTS.groupby('next_expiry'):
    secids = sorted(dfe['secid'].unique().tolist())
    if not secids:
        continue
    st = fetch_ST_for_exdate(exd, secids)
    st['next_expiry'] = exd
    st_rows.append(st)

S0_all = pd.concat(s0_rows, ignore_index=True) if s0_rows else pd.DataFrame(columns=['secid','s0_date','S0','formation_date'])
ST_all = pd.concat(st_rows, ignore_index=True) if st_rows else pd.DataFrame(columns=['secid','st_date','ST','next_expiry'])

#merges
OPTS2 = OPTS.merge(S0_all[['secid','formation_date','S0','s0_date']], on=['secid','formation_date'], how='left')
OPTS2 = OPTS2.merge(ST_all[['secid','next_expiry','ST','st_date']], on=['secid','next_expiry'],   how='left')

In [ ]:
df = OPTS2.copy()

for c in ['date','exdate','formation_date','next_expiry','s0_date','st_date']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce').dt.normalize()
#keep only the unique values drop the duplicates
if {'formation_date','date'}.issubset(df.columns) and (df['formation_date'].fillna(df['date'])==df['date']).all():
    df = df.drop(columns=['formation_date'])
if {'next_expiry','exdate'}.issubset(df.columns) and (df['next_expiry'].fillna(df['exdate'])==df['exdate']).all():
    df = df.drop(columns=['next_expiry'])

df['end_date'] = (df['st_date'] if 'st_date' in df.columns else df['exdate'])
if 'st_date' in df.columns:
    df['end_date'] = df['st_date'].where(df['st_date'].notna(), df['exdate'])

#we do not keep if it has not end_date
df = df.dropna(subset=['date','end_date']).copy()
df['days_to_expiry'] = (df['end_date'] - df['date']).dt.days

#we adapt the ken french risk free
ff2 = ff.copy()
ff2 = ff2.reset_index().rename(columns={'index':'date'}) if 'date' not in ff2.columns else ff2.reset_index()
ff2['date'] = pd.to_datetime(ff2['date'], errors='coerce').dt.normalize()

ff2['rf_daily']  = pd.to_numeric(ff2['RF'], errors='coerce') / 100.0
ff2['rf_annual'] = (1.0 + ff2['rf_daily'])**365 - 1.0
ff2 = ff2.sort_values('date')
ff2['cum_log1p'] = np.log1p(ff2['rf_daily']).cumsum()

ff_small = ff2[['date','rf_daily','rf_annual','cum_log1p']].copy()

# r_t at the formationd ay
df = df.sort_values('date')
df = pd.merge_asof(
    df,
    ff_small[['date','rf_daily','rf_annual','cum_log1p']].rename(
        columns={'rf_daily':'rf_daily_t','rf_annual':'rf_annual_t','cum_log1p':'cum_t'}
    ),
    on='date', direction='backward'
)

# r_T at the end date
right_T = ff_small[['date','cum_log1p']].rename(columns={'date':'end_date','cum_log1p':'cum_T'})
df = df.sort_values('end_date')
df = pd.merge_asof(df, right_T, on='end_date', direction='backward')
#annualized * fraction
df['rf_simple_period'] = df['rf_annual_t'] * (df['days_to_expiry'].astype(float)/365.0)

df['rf_compounded_period'] = np.expm1(df['cum_T'] - df['cum_t'])

opts2_ready = df
print(opts2_ready[['date','end_date','days_to_expiry','rf_daily_t','rf_annual_t','rf_simple_period','rf_compounded_period']].head())
opts2_ready['k'] = opts2_ready['k'] / 1000

In [ ]:
USE_WRDS = True             
IV_COL   = 'iv'              
DAY_MULT_BASE = 3            #steps per calendar day
USE_RICHARDSON = True        

#depends if you work on colab or not
try:
    from scipy.special import erf
except Exception:
    from numpy import erf
#binomial tree for finding the delta manual
@njit(fastmath=True)
def _phi(x):
    t = 1.0/(1.0+0.2316419*abs(x))
    y = 1.0 - (0.319381530*t - 0.356563782*t**2 + 1.781477937*t**3 - 1.821255978*t**4 + 1.330274429*t**5) \
              * math.exp(-0.5*x*x)/math.sqrt(2*math.pi)
    return y if x>=0 else 1.0-y

@njit(fastmath=True)
def bs_call_price(S, K, r, sigma, T):
    if T<=0:   return max(S-K, 0.0)
    if sigma<=0: return max(S - K*math.exp(-r*T), 0.0)
    srt = sigma*math.sqrt(T)
    d1  = (math.log(S/K) + (r + 0.5*sigma*sigma)*T)/srt
    d2  = d1 - srt
    return S*_phi(d1) - K*math.exp(-r*T)*_phi(d2)

@njit(fastmath=True)
def bs_call_delta(S, K, r, sigma, T):
    if T<=0 or sigma<=0:
        return 1.0 if S>K else (0.0 if S<K else 0.5)
    srt = sigma*math.sqrt(T)
    d1  = (math.log(S/K) + (r + 0.5*sigma*sigma)*T)/srt
    return _phi(d1)

@njit(fastmath=True)
def _interp1(x, xp, fp):
    n = xp.shape[0]
    if x <= xp[0]:   return fp[0]
    if x >= xp[n-1]: return fp[n-1]
    lo, hi = 0, n-1
    while hi - lo > 1:
        mid = (lo+hi)//2
        if xp[mid] <= x: lo = mid
        else:            hi = mid
    x0, x1 = xp[lo], xp[hi]
    y0, y1 = fp[lo], fp[hi]
    return y0 + (y1-y0)*(x - x0)/(x1 - x0)

@njit(fastmath=True)
def _interp_vec(xs, xp, fp, out):
    for i in range(xs.shape[0]):
        out[i] = _interp1(xs[i], xp, fp)

@njit(fastmath=True)
def _american_binomial_core(S0, K, r, sigma, T, is_call, t_div, D_div, N):
    if N < 120: N = 120
    if T <= 0.0:
        if is_call:
            return max(S0-K, 0.0), (1.0 if S0>K else (0.5 if S0==K else 0.0))
        else:
            return max(K-S0, 0.0), (-1.0 if S0<K else (-0.5 if S0==K else 0.0))

    dt   = T / N
    disc = math.exp(-r*dt)
    u    = math.exp(sigma*math.sqrt(dt))
    d    = 1.0/u
    m    = math.exp(r*dt)
    p    = (m - d) / (u - d)
    if p<0.0: p=0.0
    if p>1.0: p=1.0

    Sj = np.empty(N+1)
    for i in range(N+1):
        Sj[i] = S0 * (u**i) * (d**(N-i))
    Vn = np.maximum(Sj - K, 0.0) if is_call else np.maximum(K - Sj, 0.0)

    div_by_step = np.zeros(N+1)
    if t_div is not None and t_div.shape[0] > 0:
        for k in range(t_div.shape[0]):
            j = int(round(t_div[k]/dt))
            if j>0 and j<=N:
                div_by_step[j] += D_div[k]

    delta_root = 0.0
    for j in range(N-1, -1, -1):
        Sj  = np.empty(j+1)
        Sj1 = np.empty(j+2)
        base  = S0 * (d**j);  ratio = (u/d)
        for i in range(j+1):
            Sj[i] = base * (ratio**i)
        base1 = S0 * (d**(j+1))
        for i in range(j+2):
            Sj1[i] = base1 * (ratio**i)

        Vuse = Vn
        if div_by_step[j+1] > 0.0:
            D = div_by_step[j+1]
            Sshift = Sj1 - D
            tmp = np.empty(Sj1.shape[0])
            _interp_vec(Sshift, Sj1, Vn, tmp)
            payoff = np.maximum(Sj1 - K, 0.0) if is_call else np.maximum(K - Sj1, 0.0)
            Vuse = np.maximum(payoff, tmp)

        V = disc*(p*Vuse[1:] + (1.0-p)*Vuse[:-1])
        exer = np.maximum(Sj - K, 0.0) if is_call else np.maximum(K - Sj, 0.0)
        Vcurr = np.maximum(exer, V)

        if j == 0:
            denom = (Sj1[1] - Sj1[0])
            delta_root = (Vuse[1] - Vuse[0]) / denom if denom!=0.0 else 0.0

        Vn = Vcurr

    return float(Vn[0]), float(delta_root)

def american_price_delta(S0, K, r, sigma, T, is_call, div_schedule, day_mult=DAY_MULT_BASE):
    days = max(1, int(round(T*365.0)))
    N = max(120, days * int(day_mult))
    if div_schedule and len(div_schedule)>0:
        t_div = np.array([float(t) for (t, D) in div_schedule], dtype=np.float64)
        D_div = np.array([float(D) for (t, D) in div_schedule], dtype=np.float64)
    else:
        t_div = np.empty(0, dtype=np.float64)
        D_div = np.empty(0, dtype=np.float64)
    return _american_binomial_core(float(S0),float(K),float(r),float(sigma),float(T),bool(is_call),
                                   t_div, D_div, int(N))

def american_price_delta_extrap(S0, K, r, sigma, T, is_call, div_schedule, day_mult=DAY_MULT_BASE):
    p1, d1 = american_price_delta(S0, K, r, sigma, T, is_call, div_schedule, day_mult=day_mult)
    p2, d2 = american_price_delta(S0, K, r, sigma, T, is_call, div_schedule, day_mult=2*day_mult)
    pX, dX = 2.0*p2 - p1, 2.0*d2 - d1
    intr = max(S0-K,0.0) if is_call else max(K-S0,0.0)
    pX = max(pX, intr)
    if is_call:  dX = max(0.0, min(1.0, dX))
    else:        dX = max(-1.0, min(0.0, dX))
    return pX, dX

#pull dividends
def build_div_schedules_pair_from_wrds(df_opts, verbose=True):
    import wrds, pandas as pd
    db = wrds.Connection()

    def _cols(db, lib, tbl):
        q = f"""
        select lower(column_name) as name
        from information_schema.columns
        where table_schema='{lib}' and table_name='{tbl}'
        """
        return db.raw_sql(q)['name'].tolist()

    def _pick(cols, cands):
        for c in cands:
            if c in cols: return c
        for c in cands:
            for col in cols:
                if c in col: return col
        return None

    def _sched(series_df):
        if series_df.empty:
            return pd.Series(dtype=object)
        g = (series_df[['row_id','t_i','amount']]
             .dropna()
             .sort_values(['row_id','t_i']))
        return g.groupby('row_id').apply(
            lambda x: list(zip(x['t_i'].astype(float).tolist(),
                               x['amount'].astype(float).tolist()))
        )

    for tbl in ['distrd', 'distribution']:
        cols = _cols(db, 'optionm', tbl)
        sec = _pick(cols, ['secid','securityid','sec_id'])
        exd = _pick(cols, ['exdate','ex_date','exdt','exdivdate'])
        amt = _pick(cols, ['amount','divamt','cash_amount','div_amount','amt','amnt','cash'])
        ann = _pick(cols, ['announcedate','anndate','declare_date','declarationdate','announce_date'])
        if not (sec and exd and amt):
            continue

        ids = ",".join(map(str, sorted(df_opts['secid'].astype(int).unique().tolist())))
        s, e = df_opts['date'].min().date(), df_opts['exdate'].max().date()
        sql = (
            f"select {sec} as secid, {exd} as div_exdate, {amt} as amount" +
            (f", {ann} as announcedate" if ann else "") +
            f" from optionm.{tbl} where {sec} in ({ids}) and {exd} between '{s}' and '{e}' and {amt} > 0"
        )
        raw = db.raw_sql(sql)
        if raw.empty:
            continue

        raw.columns = [c.lower() for c in raw.columns]
        raw['div_exdate'] = pd.to_datetime(raw['div_exdate'])
        if 'announcedate' in raw.columns:
            raw['announcedate'] = pd.to_datetime(raw['announcedate'])

        X = df_opts[['row_id','secid','date','exdate']].merge(raw, on='secid', how='left')
        m = (X['div_exdate'] > X['date']) & (X['div_exdate'] <= X['exdate'])
        X = X[m].copy()
        X['t_i'] = (X['div_exdate'] - X['date']).dt.days / 365.0
        if 'announcedate' in X.columns:
            X['is_ann'] = X['announcedate'].notna() & (X['announcedate'] <= X['date'])
        else:
            X['is_ann'] = True

        sched_A = _sched(X[X['is_ann']]).rename('div_schedule_ann')
        sched_B = _sched(X).rename('div_schedule_expost')
        return sched_A, sched_B


    return pd.Series(dtype=object), pd.Series(dtype=object)


def compute_deltas_on_df(DF_IN, iv_col='iv', use_wrds=False):
    import math, numpy as np, pandas as pd

    df = DF_IN.copy()

    df['date']   = pd.to_datetime(df['date']).dt.normalize()
    df['exdate'] = pd.to_datetime(df['exdate']).dt.normalize()
    if 'cp_flag' not in df.columns:
        df['cp_flag'] = 'C'  # όλα calls
    if 'k' in df.columns and 'K' not in df.columns:
        df = df.rename(columns={'k':'K'})
    if 'days_to_expiry' in df.columns and 'T' not in df.columns:
        df['T'] = df['days_to_expiry'].astype('float64')/365.0
    if 'T' not in df.columns:
        df['T'] = (df['exdate'] - df['date']).dt.days.astype('float64')/365.0


    if 'r_eff' not in df.columns:
        if {'rf_compounded_period','T'}.issubset(df.columns):
            df['r_eff'] = np.where(df['T']>0,
                                   np.log1p(pd.to_numeric(df['rf_compounded_period'], errors='coerce'))/df['T'],
                                   0.0)
        elif 'rf_annual_t' in df.columns:
            df['r_eff'] = pd.to_numeric(df['rf_annual_t'], errors='coerce')
        else:
            df['r_eff'] = 0.0


    if use_wrds and (('div_schedule_ann' not in df.columns) or ('div_schedule_expost' not in df.columns)):
        df = df.reset_index(drop=True)
        df['row_id'] = np.arange(len(df))
        A, B = build_div_schedules_pair_from_wrds(df)
        if not A.empty: df = df.join(A, on='row_id').join(B, on='row_id')
    if 'div_schedule_ann' not in df.columns:    df['div_schedule_ann']    = [[] for _ in range(len(df))]
    if 'div_schedule_expost' not in df.columns: df['div_schedule_expost'] = [[] for _ in range(len(df))]

    for c in ['S0','K','r_eff','T', iv_col]:
        df[c] = pd.to_numeric(df[c], errors='coerce').astype('float64')

    valid = df['T'].gt(0) & df[iv_col].gt(0) \
            & df[['S0','K','r_eff','T',iv_col]].apply(np.isfinite).all(axis=1)

    def sf(x):
        try:    return float(x)
        except: return np.nan

    def _row_pricer(r):
        S0, K, T, rf, sigma = sf(r['S0']), sf(r['K']), sf(r['T']), sf(r['r_eff']), sf(r[iv_col])
        if not (np.isfinite(S0) and np.isfinite(K) and np.isfinite(T) and np.isfinite(rf) and np.isfinite(sigma)):
            return pd.Series({'price_am': np.nan, 'delta_model': np.nan})
        if T<=0.0 or sigma<=0.0:
            return pd.Series({'price_am': np.nan, 'delta_model': np.nan})

        is_call = (r['cp_flag']=='C')
        sched   = r['div_schedule_ann'] if isinstance(r['div_schedule_ann'], list) else []

        if is_call and len(sched)==0:
            return pd.Series({
                'price_am': bs_call_price(S0, K, rf, sigma, T),
                'delta_model': bs_call_delta(S0, K, rf, sigma, T)
            })
        p, d = american_price_delta_extrap(S0, K, rf, sigma, T, bool(is_call), sched,
                                           day_mult=DAY_MULT_BASE)
        return pd.Series({'price_am': p, 'delta_model': d})

    out = pd.DataFrame(index=df.index, columns=['price_am','delta_model'], dtype='float64')
    try:
        from tqdm.auto import tqdm; tqdm.pandas()
        out.loc[valid] = df.loc[valid].progress_apply(_row_pricer, axis=1)
    except Exception:
        out.loc[valid] = df.loc[valid].apply(_row_pricer, axis=1)

    df = pd.concat([df, out], axis=1)

    def _pv_div(sched, r, T):
        if not isinstance(sched, list) or len(sched)==0 or not np.isfinite(r) or not np.isfinite(T):
            return 0.0
        return float(sum(float(D)*math.exp(-float(r)*float(t)) for (t, D) in sched))
    df['pv_div_ann'] = [ _pv_div(v, rr, TT) for v, rr, TT in zip(df['div_schedule_ann'], df['r_eff'], df['T']) ]

    def _bs_delta_vec(S, K, r, sig, T, is_call):
        S = np.asarray(S, float); K=np.asarray(K, float); r=np.asarray(r, float)
        sig=np.asarray(sig, float); T=np.asarray(T, float); is_call=np.asarray(is_call, bool)
        out = np.full_like(S, np.nan, float)
        ok = (T>0) & (sig>0) & np.isfinite(S) & np.isfinite(K) & np.isfinite(r)
        if ok.any():
            srt = sig[ok]*np.sqrt(T[ok])
            d1  = (np.log(S[ok]/K[ok]) + (r[ok] + 0.5*sig[ok]**2)*T[ok]) / srt
            phi = 0.5*(1.0 + erf(d1/np.sqrt(2.0)))
            out[ok] = np.where(is_call[ok], phi, phi - 1.0)
        bd = ~ok & np.isfinite(S) & np.isfinite(K)
        if bd.any():
            oc = is_call[bd]; SgK = S[bd]>K[bd]; SlK = S[bd]<K[bd]; eq = ~(SgK|SlK)
            tmp = np.empty(bd.sum(), float)
            tmp[ oc & SgK] =  1.0; tmp[ oc & SlK] =  0.0; tmp[ oc & eq] =  0.5
            tmp[~oc & SlK] = -1.0; tmp[~oc & SgK] =  0.0; tmp[~oc & eq] = -0.5
            out[bd] = tmp
        return out

    def _bs_delta_div_vec(S,K,r,sig,T,pv,is_call):
        S = np.asarray(S, float); K=np.asarray(K, float); pv=np.asarray(pv, float)
        is_call = np.asarray(is_call, bool)
        S_eff = np.where(is_call, np.maximum(S - pv, 1e-12), S)
        K_eff = np.where(is_call, K, K + pv)
        return _bs_delta_vec(S_eff, K_eff, r, sig, T, is_call)

    is_call_mask = df['cp_flag'].eq('C').values
    df['delta_bs_plain']  = _bs_delta_vec( df['S0'].values, df['K'].values, df['r_eff'].values,
                                           df[iv_col].values, df['T'].values, is_call_mask )
    df['delta_bs_divadj'] = _bs_delta_div_vec( df['S0'].values, df['K'].values, df['r_eff'].values,
                                               df[iv_col].values, df['T'].values, df['pv_div_ann'].values,
                                               is_call_mask )

    return df



DF_IN = opts2_ready.copy()     
if 'cp_flag' not in DF_IN.columns:
    DF_IN['cp_flag'] = 'C'
opts_with_deltas = compute_deltas_on_df(DF_IN, iv_col=IV_COL, use_wrds=USE_WRDS)

In [ ]:

df = opts_with_deltas.copy()

#numerics & safe fallbacks
num_cols = ['S0','K','mid','price_am','delta_model','delta_bs_plain','delta_bs_divadj']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

#days
if 'days_to_expiry' in df.columns:
    df['days'] = pd.to_numeric(df['days_to_expiry'], errors='coerce')
elif 'T' in df.columns:
    df['days'] = pd.to_numeric(df['T'], errors='coerce')*365.0
else:
    df['days'] = (pd.to_datetime(df['exdate']) - pd.to_datetime(df['date'])).dt.days

#moneyness
df['logm'] = np.log(df['S0'] / df['K'])

#differences vs baselines
df['d_model_vs_bs_plain']   = df['delta_model'] - df['delta_bs_plain']
df['d_model_vs_bs_divadj']  = df['delta_model'] - df['delta_bs_divadj']

#price basis (model American vs mid)
df['price_basis_pct'] = (df['price_am'] - df['mid']) / df['mid']
df['basis_bps']       = 1e4 * (df['price_am'] - df['mid']) / df['mid'].clip(lower=1e-6)

mny_bins = [-0.4,-0.2,-0.1,-0.05,-0.02,0,0.02,0.05,0.1,0.2,0.4]
day_bins = [0,30,60,91,122,152,182,273,365,9999]
df['mny_bucket'] = pd.cut(df['logm'], mny_bins)
df['T_bucket']   = pd.cut(df['days'], day_bins, right=True)

def summarize_series(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors='coerce').dropna()
    if s.empty:
        return pd.Series({'N':0,'mean':np.nan,'MAE':np.nan,'RMSE':np.nan,'p10':np.nan,'p50':np.nan,'p90':np.nan})
    return pd.Series({
        'N':   s.size,
        'mean': s.mean(),
        'MAE':  s.abs().mean(),
        'RMSE': np.sqrt((s**2).mean()),
        'p10':  s.quantile(0.10),
        'p50':  s.quantile(0.50),
        'p90':  s.quantile(0.90),
    })

def corr_safe(a, b):
    t = pd.DataFrame({'a':a, 'b':b}).dropna()
    return t['a'].corr(t['b']) if len(t)>=2 else np.nan

print("=== COVERAGE ===")
print(f"N rows: {len(df):,}")
print(f"delta_model coverage: {df['delta_model'].notna().mean():.2%}")
print(f"days_to_expiry (min/median/max): {df['days'].min()} / {df['days'].median()} / {df['days'].max()}")
print()

print("=== OVERALL (Δ_model − Δ_BS_plain) ===")
s1 = summarize_series(df['d_model_vs_bs_plain'])
s1['corr(model, BS_plain)'] = corr_safe(df['delta_model'], df['delta_bs_plain'])
display(pd.DataFrame([s1]))

print("\n=== OVERALL (Δ_model − Δ_BS_divadj) ===")
s2 = summarize_series(df['d_model_vs_bs_divadj'])
s2['corr(model, BS_divadj)'] = corr_safe(df['delta_model'], df['delta_bs_divadj'])
display(pd.DataFrame([s2]))

print("\n=== BY LOG-MONEYNESS (Δ_model − Δ_BS_divadj) ===")
g_mny = df.groupby('mny_bucket', observed=True)['d_model_vs_bs_divadj'].apply(summarize_series).unstack()
display(g_mny)

print("\n=== BY DAYS TO EXPIRY (Δ_model − Δ_BS_divadj) ===")
g_T = df.groupby('T_bucket', observed=True)['d_model_vs_bs_divadj'].apply(summarize_series).unstack()
display(g_T)

print("\n=== BASIS bps (all) ===")
display(df['basis_bps'].describe(percentiles=[.1,.5,.9,.99]).to_frame('basis_bps'))

print("\n=== BASIS bps (liquidity-filtered) ===")
if {'bid','ask'}.issubset(df.columns):
    liq = (df['mid']>=0.5) & ((df['ask'] - df['bid']) <= 0.5)
else:
    liq = (df['mid']>=0.5)
display(df.loc[liq, 'basis_bps'].describe(percentiles=[.1,.5,.9,.99]).to_frame('basis_bps_liq'))

if 'has_div_future' in df.columns:
    print("\n=== BY has_div_future (ex-ante baseline, Δ_model − Δ_BS_divadj) ===")
    rows = []
    for flag, sub in [('True', df[df['has_div_future']]), ('False', df[~df['has_div_future']])]:
        s = summarize_series(sub['d_model_vs_bs_divadj'])
        s['label'] = f"has_div_future={flag}"
        rows.append(s)
    display(pd.DataFrame(rows))

if 'unknown_div_flag' in df.columns:
    print("\n=== Unknown dividends group (has_div_future=True & has_div_ann=False) ===")
    unk = df[df['unknown_div_flag']]
    if len(unk):
        u = summarize_series(unk['d_model_vs_bs_divadj'])
        u['corr(model, BS_divadj)'] = corr_safe(unk['delta_model'], unk['delta_bs_divadj'])
        display(pd.DataFrame([u]))
    else:
        print("None")

In [ ]:
def _num(x):
    return pd.to_numeric(x, errors='coerce')

def _asfloat_arr(x):
    return np.asarray(pd.to_numeric(x, errors='coerce'), dtype=float)

def _canon_ticker(s):
    return re.sub(r'[.\-]', '', str(s).upper()).strip()

# ---------- RF: compounded period from daily FF ----------
def add_rf_period_from_ff(df_opts: pd.DataFrame, ff_daily: pd.DataFrame,
                          start_col='date', end_col=None, rf_col='RF',
                          rf_is_percent=True, out_col='rf_compounded_period'):

    df = df_opts.copy()
    if end_col is None:
        end_col = 'st_date' if 'st_date' in df.columns else 'exdate'

    ff = ff_daily[['date', rf_col]].copy()
    ff['date'] = pd.to_datetime(ff['date']).dt.normalize()
    rf = _num(ff[rf_col])
    if rf_is_percent:
        rf = rf / 100.0
    ff['log1p_rf'] = np.log1p(rf)
    ff['cum_log']  = ff['log1p_rf'].cumsum()

    df[start_col] = pd.to_datetime(df[start_col]).dt.normalize()
    df[end_col]   = pd.to_datetime(df[end_col]).dt.normalize()

    s = pd.merge_asof(df[[start_col]].sort_values(start_col),
                      ff[['date','cum_log']].rename(columns={'date':'_d'}),
                      left_on=start_col, right_on='_d', direction='backward')
    e = pd.merge_asof(df[[end_col]].sort_values(end_col),
                      ff[['date','cum_log']].rename(columns={'date':'_d'}),
                      left_on=end_col, right_on='_d', direction='backward')
    s = s['cum_log'].reindex(df.index).fillna(method='ffill').fillna(0.0).values
    e = e['cum_log'].reindex(df.index).fillna(method='ffill').fillna(0.0).values

    gross = np.exp(e - s)           #product of (1+RF)
    df[out_col] = gross - 1.0       #period RF
    return df

#atm option choise 1 per stock
def pick_atm_one_per_stock(df: pd.DataFrame, mny_low=0.8, mny_high=1.2) -> pd.DataFrame:
    X = df.copy()
    X['ticker_norm'] = X['ticker'].map(_canon_ticker)
    X['date']   = pd.to_datetime(X['date']).dt.normalize()
    X['exdate'] = pd.to_datetime(X['exdate']).dt.normalize()
    X['mny']    = _num(X['S0']) / _num(X['K'])

    atm = X[X['mny'].between(mny_low, mny_high)].copy()
    atm['dist']      = (atm['mny'] - 1.0).abs()
    atm['is_otm_pr'] = (atm['mny'] <= 1.0).astype(int)
    atm = (atm.sort_values(['date','ticker_norm','dist','is_otm_pr'],
                           ascending=[True, True, True, False])
             .groupby(['date','ticker_norm'], as_index=False).head(1))
    return atm.drop(columns=['dist','is_otm_pr'])

#delta hedge returns
def _pick_rf_period(df):
    for c in ['rf_compounded_period','RF_period','rf_period','rf_monthly','rf']:
        if c in df.columns:
            return _asfloat_arr(df[c])
    return np.zeros(len(df), dtype=float)

def dh_return_eq3(S0, ST, K, Delta, C0, rf, eps=1e-12):
    N = Delta*ST - np.maximum(ST - K, 0.0)
    D = Delta*S0 - C0
    with np.errstate(divide='ignore', invalid='ignore'):
        r = N/D - 1.0 - rf
    r[(~np.isfinite(r)) | (D<=eps)] = np.nan
    return r, D

def dh_return_tc_eq4_from_noTC(r_no_tc, rf, D, D_TC):
    # (1 + r_no_tc + rf) * (D / D_TC) - 1 - rf
    with np.errstate(divide='ignore', invalid='ignore'):
        gross_tc = (1.0 + r_no_tc + rf) * (D / D_TC)
        r_tc = gross_tc - 1.0 - rf
    r_tc[(~np.isfinite(r_tc)) | (D_TC<=0)] = np.nan
    return r_tc

def compute_static_dh(df: pd.DataFrame,
                      delta_col='delta_model',
                      C_col='mid',          
                      eff_frac=0.5,         
                      side='sell',         
                      eps=1e-12) -> pd.DataFrame:

    out = df.copy()
    out['date']   = pd.to_datetime(out['date']).dt.normalize()
    out['exdate'] = pd.to_datetime(out['exdate']).dt.normalize()

    S0, ST, K  = _asfloat_arr(out['S0']), _asfloat_arr(out['ST']), _asfloat_arr(out['K'])
    Delta      = _asfloat_arr(out[delta_col])
    C0         = _asfloat_arr(out[C_col])
    rf         = _pick_rf_period(out)

    r_no, D = dh_return_eq3(S0, ST, K, Delta, C0, rf, eps=eps)
    out['r_dh_no_tc']  = r_no
    out['denom_no_tc'] = D

    if {'bid','ask'}.issubset(out.columns) and (eff_frac is not None):
        half_spread = 0.5*np.maximum(_asfloat_arr(out['ask']) - _asfloat_arr(out['bid']), 0.0)
        spread_eff  = float(eff_frac) * half_spread
        C0_TC = (C0 - spread_eff) if side=='sell' else (C0 + spread_eff)
        D_TC  = Delta*S0 - C0_TC
        out['r_dh_tc']  = dh_return_tc_eq4_from_noTC(r_no, rf, D, D_TC)
        out['denom_tc'] = D_TC

    #doi weight
    out['doi'] = _num(out['open_interest']) * _num(out['mid']) if 'open_interest' in out.columns else np.nan
    return out

def aggregate_monthly_returns(df_with_r: pd.DataFrame, use_tc=False) -> pd.DataFrame:
    col = 'r_dh_tc' if use_tc else 'r_dh_no_tc'
    g = df_with_r.loc[df_with_r[col].notna(), ['date', col, 'doi']].copy()
    g['date'] = pd.to_datetime(g['date']).dt.normalize()
    g['doi']  = _num(g['doi']).clip(lower=0)

    ew  = g.groupby('date', observed=True)[col].mean().rename('EW')
    sW  = g.groupby('date', observed=True)['doi'].sum().rename('sum_doi')
    g   = g.join(sW, on='date')
    g['w']  = np.where(g['sum_doi']>0, g['doi']/g['sum_doi'], np.nan)
    g['wx'] = g['w']*g[col]
    oiw = g.groupby('date', observed=True)['wx'].sum().rename('OIW')
    N   = g.groupby('date', observed=True)[col].size().rename('N')
    out = pd.concat([ew, oiw, N], axis=1).reset_index()
    out['use_tc'] = bool(use_tc)
    return out

def _wstats_month(x, w=None):
    x = _num(x).dropna()
    if x.empty: return np.nan, np.nan
    if w is None:
        return float(x.mean()), float(x.std(ddof=0))
    w = _num(w).reindex_like(x).fillna(0)
    if (w<=0).all(): return np.nan, np.nan
    wn = w / w.sum()
    mu = float((x*wn).sum())
    sd = float(np.sqrt(((x-mu)**2 * wn).sum()))
    return mu, sd

def _ts_avg_quantiles(groups, col):
    mins=[]; q1s=[]; meds=[]; q3s=[]; maxs=[]
    for _, g in groups:
        x = _num(g[col]).dropna()
        if x.empty: continue
        mins.append(float(x.min()))
        q1s.append(float(x.quantile(0.25)))
        meds.append(float(x.quantile(0.50)))
        q3s.append(float(x.quantile(0.75)))
        maxs.append(float(x.max()))
    if not mins: return [np.nan]*5
    return [np.mean(mins), np.mean(q1s), np.mean(meds), np.mean(q3s), np.mean(maxs)]

def build_table_I_like(opt_r: pd.DataFrame, use_tc=False, delta_col='delta_model'):
    df = opt_r.copy()
    df['date'] = pd.to_datetime(df['date']).dt.normalize()

    rcol = 'r_dh_tc' if use_tc else 'r_dh_no_tc'
    df = df[df[rcol].notna()].copy()  # ίδιο sample για όλες τις γραμμές
    df['opt_ret_pct'] = 100.0 * _num(df[rcol])

    df['stock_ret_pct'] = 100.0 * (_num(df['ST'])/_num(df['S0']) - 1.0)
    df['days']          = (pd.to_datetime(df['exdate']) - pd.to_datetime(df['date'])).dt.days
    df['mny_S_over_K']  = _num(df['S0'])/_num(df['K'])
    df['Delta_use']     = _num(df[delta_col]) if (delta_col in df.columns) else _num(df.get('delta', np.nan))

    #greeks scaling
    S = _num(df['S0'])
    df['Gamma_Tbl'] = _num(df.get('gamma', np.nan)) * S    
    df['Vega_Tbl']  = _num(df.get('vega',  np.nan)) / S
    df['Theta_Tbl'] = _num(df.get('theta', np.nan)) / S

    df['qspread_pct'] = 100.0 * (_num(df['ask']) - _num(df['bid'])) / _num(df['mid'])

    #per-month stats
    groups = df.groupby('date', observed=True)
    ew_means = {lab: [] for lab in ['opt_ret_pct','stock_ret_pct','days','mny_S_over_K','Delta_use','Gamma_Tbl','Vega_Tbl','Theta_Tbl','qspread_pct']}
    ew_sds   = {lab: [] for lab in ew_means}
    oiw_means= {lab: [] for lab in ew_means}
    oiw_sds  = {lab: [] for lab in ew_means}

    for d, g in groups:
        w = _num(g['doi']).clip(lower=0)
        for lab in ew_means.keys():
            mu_ew, sd_ew   = _wstats_month(g[lab], w=None)
            mu_oiw, sd_oiw = _wstats_month(g[lab], w=w)
            ew_means[lab].append(mu_ew); ew_sds[lab].append(sd_ew)
            oiw_means[lab].append(mu_oiw); oiw_sds[lab].append(sd_oiw)

    def ts_avg(series_dict):
        return {k: np.nanmean(v) if len(v) else np.nan for k,v in series_dict.items()}

    mean_OIW = ts_avg(oiw_means); mean_EW  = ts_avg(ew_means)
    sd_OIW   = ts_avg(oiw_sds);   sd_EW    = ts_avg(ew_sds)

    #unweighted monthly quantiles/min/max 
    Min,Q1,Q2,Q3,Max = {},{},{},{},{}
    for lab in ew_means.keys():
        m,q1,q2,q3,M = _ts_avg_quantiles(groups, lab)
        Min[lab]=m; Q1[lab]=q1; Q2[lab]=q2; Q3[lab]=q3; Max[lab]=M

    Count = np.mean(groups.size().values) if len(groups) else np.nan

    def row(lab):
        return [mean_OIW[lab], mean_EW[lab], sd_OIW[lab], sd_EW[lab],
                Min[lab], Q1[lab], Q2[lab], Q3[lab], Max[lab], Count]

    idx = ['Option return (%)','Stock return (%)','Days to maturity','Moneyness (S/K)',
           'Delta','Gamma','Vega','Theta','Quoted spread (%)']
    rows = [
        row('opt_ret_pct'),
        row('stock_ret_pct'),
        row('days'),
        row('mny_S_over_K'),
        row('Delta_use'),
        row('Gamma_Tbl'),
        row('Vega_Tbl'),
        row('Theta_Tbl'),
        row('qspread_pct'),
    ]
    cols = pd.MultiIndex.from_tuples(
        [('Mean','OIW'), ('Mean','EW'), ('Sd','OIW'), ('Sd','EW'),
         ('','Min'),('','Q1'),('','Q2'),('','Q3'),('','Max'),('','Count')]
    )
    tbl = pd.DataFrame(rows, index=idx, columns=cols)
    return tbl


In [ ]:
atm = pick_atm_one_per_stock(opts_with_deltas, mny_low=0.8, mny_high=1.2)
opt_r = compute_static_dh(atm, delta_col='delta_model', C_col='mid', eff_frac=0.5, side='sell')
rets_no = aggregate_monthly_returns(opt_r, use_tc=False)
rets_tc = aggregate_monthly_returns(opt_r, use_tc=True)

table_no = build_table_I_like(opt_r, use_tc=False, delta_col='delta_model')
table_tc = build_table_I_like(opt_r, use_tc=True,  delta_col='delta_model') 

print(table_no.round(2))
print(table_tc.round(2))


In [ ]:
pd.set_option("display.max_columns", None)
surf_df = pd.read_parquet('/content/drive/MyDrive/Dissertation/vol_filt_5_22.parquet') #dataset not available on github but information how to obtain it from wrds availabe on read.me file

In [ ]:
from typing import Dict, List, Optional, Tuple

GRID_DAYS_DEFAULT   = [30, 60, 91, 122, 152, 182, 273, 365, 547, 730] 
GRID_DELTAS_DEFAULT = [d for d in range(-50, 51, 5) if d != 0] 

def _to_month_start(s) -> pd.Series:
    s = pd.to_datetime(s)
    return s.dt.to_period('M').dt.to_timestamp()

def _nearest_index_vectorized(values: np.ndarray, grid: List[int]) -> np.ndarray:
    g = np.asarray(grid, dtype=float)
    v = np.asarray(values, dtype=float)
    return np.abs(v[:, None] - g[None, :]).argmin(axis=1).astype(np.int64)

def _auto_grid_deltas(surf_df: pd.DataFrame) -> List[int]:
    try:
        vals = (surf_df['delta'].round().astype(int)
                .value_counts().nlargest(18).index.tolist())
        vals = sorted(vals)
        if len(vals) == 18:
            return vals
    except Exception:
        pass
    return GRID_DELTAS_DEFAULT

#normalize and create the import dataset for the cnn
@dataclass
class GridScaler:
    mu: np.ndarray  
    sd: np.ndarray  
    def transform(self, X: np.ndarray) -> np.ndarray:
        return (X - self.mu) / np.where(self.sd==0, 1.0, self.sd)

def build_cnn_dataset_inram(
    surf_df: pd.DataFrame,
    opt_r: pd.DataFrame,
    grid_days: Optional[List[int]] = None,
    grid_deltas: Optional[List[int]] = None,
    dtype_X: np.dtype = np.float16,
    add_controls: bool = True,
    verbose: bool = True
) -> Dict[str, object]:
    grid_days   = GRID_DAYS_DEFAULT if grid_days is None else grid_days
    grid_deltas = _auto_grid_deltas(surf_df) if grid_deltas is None else grid_deltas

    S = surf_df[['secid','date','month','days','delta','impl_volatility']].copy()
    S['date']  = pd.to_datetime(S['date']).dt.normalize()
    S['month'] = _to_month_start(S['month'])
    S['secid'] = pd.to_numeric(S['secid'], errors='coerce').astype('Int64')

    if not S['days'].isin(grid_days).all():
        S['days'] = pd.to_numeric(S['days'], errors='coerce').astype(float).round().astype(int)
    S = S[S['days'].isin(grid_days)]

    P = opt_r.copy()
    for c in ['secid','month','s0_date']:
        if c not in P.columns:
            raise KeyError(f"Λείπει στήλη '{c}' από το opt_r.")
    P['month']   = _to_month_start(P['month'])
    P['s0_date'] = pd.to_datetime(P['s0_date']).dt.normalize()
    P['secid']   = pd.to_numeric(P['secid'], errors='coerce').astype('Int64')

    if 'doi' not in P.columns and {'open_interest','mid'}.issubset(P.columns):
        P['doi'] = pd.to_numeric(P['open_interest'], errors='coerce') * pd.to_numeric(P['mid'], errors='coerce')


    P = (P.sort_values(['secid','month'])
           .drop_duplicates(subset=['secid','month'], keep='first')
           .reset_index(drop=True))

    J = S.merge(P[['secid','month','s0_date']], on=['secid','month'], how='inner', copy=False)
    J['dd']  = (J['date'] - J['s0_date']).dt.days.astype('int32')
    #keep the closest day
    J['ord'] = np.where(J['dd']>=0, J['dd'].astype('int64'), 10**9 + np.abs(J['dd']).astype('int64'))
    idx_min  = J.groupby(['secid','month'])['ord'].idxmin()
    chosen   = J.loc[idx_min, ['secid','month','date']]
    S_use    = S.merge(chosen, on=['secid','month','date'], how='inner')
    if verbose:
        print(f"Surface επιλογές: {len(S_use):,} rows από {len(P):,} (secid,month).")

    if not S_use['delta'].isin(grid_deltas).all():
        jid = _nearest_index_vectorized(S_use['delta'].to_numpy(), grid_deltas)
    else:
        map_del = {int(d):j for j,d in enumerate(grid_deltas)}
        jid = S_use['delta'].map(map_del).to_numpy(dtype=np.int64)

    iid = _nearest_index_vectorized(S_use['days'].to_numpy(), grid_days)

    keys = P[['secid','month']].copy()
    keys['row_id'] = np.arange(len(keys), dtype=np.int64)
    S_use = S_use.merge(keys, on=['secid','month'], how='left')
    rid = S_use['row_id'].to_numpy(dtype=np.int64)

    N = len(keys)
    sums = np.zeros((N, len(grid_days), len(grid_deltas)), dtype=np.float32)
    cnts = np.zeros_like(sums, dtype=np.uint16)
    vv   = pd.to_numeric(S_use['impl_volatility'], errors='coerce').to_numpy(dtype=np.float32)

    np.add.at(sums, (rid, iid, jid), vv)
    np.add.at(cnts, (rid, iid, jid), 1)

    A = sums / np.where(cnts>0, cnts, np.nan)  
    valid_rows = np.isfinite(A).all(axis=(1,2))
    if verbose:
        bad = (~valid_rows).sum()
        print(f"Full surfaces: {valid_rows.sum():,} / {N:,}  (drop {bad:,} rows no full 10×18)")

    A = A[valid_rows]
    keys = keys.loc[valid_rows].reset_index(drop=True)
    P_use = P.merge(keys[['secid','month']], on=['secid','month'], how='inner')

    #build the meta values for transfering
    meta_cols_base = ['secid','month','s0_date','doi','r_dh_no_tc','r_dh_tc']
    extra_cols = [c for c in ['mid','bid','ask','iv','delta','gamma','vega','theta','S0','K','T','rf_annual_t']
                  if c in P_use.columns]
    meta = P_use[meta_cols_base + extra_cols].reset_index(drop=True)

    #surface based controls
    if add_controls:
        def _nearest_idx(vals, x):
            arr = np.asarray(vals, float); return int(np.argmin(np.abs(arr - x)))
        i30, i365 = _nearest_idx(grid_days,30), _nearest_idx(grid_days,365)
        jc50, jp50, jp20 = _nearest_idx(grid_deltas,50), _nearest_idx(grid_deltas,-50), _nearest_idx(grid_deltas,-20)
        A30c = A[:, i30, jc50]; A30p = A[:, i30, jp50]
        iv_atm_30  = 0.5*(A30c + A30p)
        iv_atm_365 = 0.5*(A[:, i365, jc50] + A[:, i365, jp50]) * 0.5 * 2  # (ίδιο αποτέλεσμα)
        meta['IVolATM']   = iv_atm_30.astype(np.float32)
        meta['CP_Spread'] = (A30c - A30p).astype(np.float32)
        meta['IVSlope']   = (iv_atm_365 - iv_atm_30).astype(np.float32)
        meta['IVolSkew']  = (A[:, i30, jp20] - A30c).astype(np.float32)

    #xreate cnn input
    X_raw = A.astype(dtype_X)[:, None, :, :]             
    years = pd.to_datetime(meta['month']).dt.year
    start_year      = int(years.min())
    first_test_year = start_year + 7
    train_mask_init = years.between(start_year, first_test_year-1)
    if train_mask_init.sum() == 0:
        raise RuntimeError("no 7 years window")

    mu = X_raw[train_mask_init].astype(np.float32).mean(axis=0, keepdims=False)
    sd = X_raw[train_mask_init].astype(np.float32).std(axis=0, ddof=1, keepdims=False)
    scaler = GridScaler(mu=mu, sd=np.where(sd==0, 1.0, sd).astype(np.float32))

    # scaled X with float 32
    Xz = scaler.transform(X_raw.astype(np.float32)).astype(np.float32)

    #expanding splits
    last_test_year = int(years.max())
    splits: Dict[int, Dict[str, np.ndarray]] = {}
    for y in range(first_test_year, last_test_year+1):
        tr = years <= (y-1)
        te = years == y
        if te.sum() == 0 or tr.sum() == 0:
            continue
        splits[y] = dict(
            train_idx = np.where(tr)[0],
            test_idx  = np.where(te)[0],
            weights_test = pd.to_numeric(meta.loc[te,'doi'], errors='coerce').astype(np.float32).to_numpy(),
            secid_test   = meta.loc[te,'secid'].to_numpy(),
            month_test   = pd.to_datetime(meta.loc[te,'month']).to_numpy('datetime64[M]')
        )

    return dict(
        X_raw=X_raw, Xz=Xz, meta=meta, scaler=scaler,
        grid_days=grid_days, grid_deltas=grid_deltas,
        start_year=start_year, first_test_year=first_test_year, last_test_year=last_test_year,
        splits=splits
    )

def export_year_npz(X: np.ndarray, y: np.ndarray, idx: np.ndarray, out_path: str):
    Xb = X[idx].astype(np.float32, copy=False)
    yb = y[idx].astype(np.float32, copy=False)
    np.savez_compressed(out_path, X=Xb, y=yb)


In [ ]:
res = build_cnn_dataset_inram(surf_df, opt_r, dtype_X=np.float16, add_controls=True, verbose=True)

Xz   = res['Xz']        
meta = res['meta']      
spl  = res['splits']

In [ ]:
import json
from datetime import datetime

EXPORT_DIR = "/content/drive/MyDrive/Dissertation/data"  #colab directories
os.makedirs(EXPORT_DIR, exist_ok=True)

xz_path = os.path.join(EXPORT_DIR, "Xz_float32.npy")
np.save(xz_path, Xz.astype(np.float32, copy=False))
#parquet for minimal space 
meta_path = os.path.join(EXPORT_DIR, "meta.parquet")
meta.to_parquet(meta_path, index=False)

if 'scaler' in res:
    sc_path = os.path.join(EXPORT_DIR, "grid_scaler_init7y.npz")
    np.savez_compressed(sc_path, mu=res['scaler'].mu, sd=res['scaler'].sd)
else:
    sc_path = None

splits_json = {}
for y, pack in spl.items():
    splits_json[str(y)] = {
        'train_idx': np.asarray(pack['train_idx']).tolist(),
        'test_idx' : np.asarray(pack['test_idx']).tolist()
    }
sp_path = os.path.join(EXPORT_DIR, "splits.json")
with open(sp_path, "w") as f:
    json.dump(splits_json, f)

#we can remember what we saved
manifest = {
    "created_at_utc": datetime.utcnow().isoformat() + "Z",
    "Xz_shape": tuple(map(int, Xz.shape)),
    "rows_meta": int(len(meta)),
    "oos_years": sorted(map(int, spl.keys())),
    "paths": {"Xz": xz_path, "meta": meta_path, "scaler": sc_path, "splits": sp_path}
}
with open(os.path.join(EXPORT_DIR, "manifest.json"), "w") as f:
    json.dump(manifest, f, indent=2)


In [ ]:
from pprint import pprint


def ensure_month_doi(meta: pd.DataFrame) -> pd.DataFrame:
    df = meta.copy()
    if 'month' in df:
        m = pd.to_datetime(df['month'], errors='coerce')
    elif 's0_date' in df:
        m = pd.to_datetime(df['s0_date'], errors='coerce')
    else:
        raise KeyError("Need 'month' or 's0_date' to build calendar month.")
    df['month'] = m.dt.to_period('M').dt.to_timestamp()
    #doi weight
    if 'doi' not in df or df['doi'].isna().all():
        if {'open_interest','mid'}.issubset(df.columns):
            df['doi'] = pd.to_numeric(df['open_interest'], errors='coerce') * pd.to_numeric(df['mid'], errors='coerce')
        else:
            df['doi'] = 1.0
    df['doi'] = pd.to_numeric(df['doi'], errors='coerce').fillna(0.0).clip(lower=0)
    return df

def load_splits(res=None, json_path=None):
    if res is not None and 'splits' in res:
        return res['splits']
    if json_path:
        import json
        with open(json_path,'r') as f: sj = json.load(f)
        return {int(y): {'train_idx': np.asarray(v['train_idx'], int),
                         'test_idx' : np.asarray(v['test_idx'],  int)} for y,v in sj.items()}
    raise ValueError("Pass res['splits'] or a path to splits.json")

#Pca espanding
try:
    from sklearn.decomposition import PCA as _SkPCA
    _HAS_SK = True
except Exception:
    _HAS_SK = False

class FittedPCA:
    def __init__(self, mean_, comps_, evr_):
        self.mean_ = mean_; self.components_ = comps_; self.evr_ = evr_

def _fit_pca_np(X, k):
    mu = X.mean(axis=0); Xc = X - mu
    U,S,Vt = np.linalg.svd(Xc, full_matrices=False)
    evr = (S**2)/(S**2).sum()
    return FittedPCA(mu, Vt[:k,:], evr[:k])

def _fit_pca(X, k):
    if _HAS_SK:
        p = _SkPCA(n_components=k, svd_solver='full', random_state=0).fit(X)
        return FittedPCA(p.mean_.copy(), p.components_.copy(), p.explained_variance_ratio_.copy())
    return _fit_pca_np(X, k)

def _pca_scores(model, X): return (X - model.mean_) @ model.components_.T

def compute_pcs_expanding(Xz, meta_df, splits, var_threshold=0.99, k_max=8, align_sign=True):
    N = Xz.shape[0]; X = Xz.reshape(N,-1).astype(np.float32)
    first_y = sorted(splits.keys())[0]; tr0 = splits[first_y]['train_idx']
    # K from first train
    pfull = _fit_pca(X[tr0], min(k_max, X.shape[1]))
    K = int(np.searchsorted(np.cumsum(pfull.evr_), var_threshold) + 1)
    K = max(1, min(K, k_max))
    pref = _fit_pca(X[tr0], K); ref = pref.components_.copy()

    scores = np.full((N,K), np.nan, np.float32); models={}
    for y in sorted(splits.keys()):
        tr, te = splits[y]['train_idx'], splits[y]['test_idx']
        pj = _fit_pca(X[tr], K)
        if align_sign:
            for j in range(K):
                if np.dot(pj.components_[j], ref[j]) < 0: pj.components_[j] *= -1.0
        models[y] = pj
        scores[tr,:] = _pca_scores(pj, X[tr])
        scores[te,:] = _pca_scores(pj, X[te])

    meta_pca = meta_df.copy()
    for j in range(K): meta_pca[f'PC{j+1}'] = scores[:,j]
    info = {'K': K, 'cumvar_first_train': float(np.cumsum(pref.evr_)[-1])}
    return meta_pca, models, info

#wls baseline
def _standardize_train_test(Xtr, Xte):
    mu = np.nanmean(Xtr, axis=0)
    sd = np.nanstd(Xtr, axis=0, ddof=0)
    sd[~np.isfinite(sd) | (sd==0)] = 1.0
    return (Xtr-mu)/sd, (Xte-mu)/sd

def _ols_wls(Xtr, ytr, wtr, Xte, ridge=1e-4):
    Xtr_ = np.c_[np.ones(len(Xtr)), Xtr]; Xte_ = np.c_[np.ones(len(Xte)), Xte]
 
    w = np.asarray(wtr, float); w[~np.isfinite(w) | (w<0)] = 0.0
    if w.sum() <= 0:  #fallback EW
        w = np.ones_like(w)
    W = np.diag(w)
    A = Xtr_.T @ W @ Xtr_ + ridge*np.eye(Xtr_.shape[1])
    b = Xtr_.T @ W @ ytr
    try:
        beta = np.linalg.solve(A,b)
    except np.linalg.LinAlgError:
        beta = np.linalg.lstsq(A,b,rcond=None)[0]
    return Xtr_@beta, Xte_@beta

def run_pca_baseline(meta_pca, splits, target_col='r_dh_no_tc', weight_col='doi'):
    pc_cols = [c for c in meta_pca.columns if c.startswith('PC')]
    if not pc_cols:
        raise RuntimeError("No PC columns found in meta_pca.")
    y_hat = np.full(len(meta_pca), np.nan, np.float32)
    wmse_num = wmse_den = emse_num = emse_den = 0.0

    for y in sorted(splits.keys()):
        tr, te = splits[y]['train_idx'], splits[y]['test_idx']
        df_tr = meta_pca.iloc[tr].copy()
        df_te = meta_pca.iloc[te].copy()
        
        mask_tr = np.isfinite(df_tr[target_col].to_numpy(float))
        for c in pc_cols: mask_tr &= np.isfinite(df_tr[c].to_numpy(float))
        mask_tr &= (pd.to_numeric(df_tr[weight_col], errors='coerce').to_numpy(float) >= 0)
        df_tr = df_tr.loc[mask_tr]
        if df_tr.empty or len(df_te)==0:
            continue

        Xtr = df_tr[pc_cols].to_numpy(float)
        ytr = df_tr[target_col].to_numpy(float)
        wtr = pd.to_numeric(df_tr[weight_col], errors='coerce').to_numpy(float)

        Xte = df_te[pc_cols].to_numpy(float)
        Xtr, Xte = _standardize_train_test(Xtr, Xte)
        
        _, yhat_te = _ols_wls(Xtr, ytr, wtr, Xte, ridge=1e-4)
        y_hat[te] = yhat_te

        #metrics
        yte = pd.to_numeric(df_te[target_col], errors='coerce').to_numpy(float)
        wte = pd.to_numeric(df_te[weight_col], errors='coerce').to_numpy(float)
        m = np.isfinite(yte) & np.isfinite(yhat_te)
        wmse_num += np.nansum(wte[m]*(yhat_te[m]-yte[m])**2); wmse_den += np.nansum(wte[m])
        emse_num += np.nansum((yhat_te[m]-yte[m])**2);       emse_den += np.sum(m)

    metrics = {
        'OOS_WMSE': float(wmse_num / wmse_den) if wmse_den>0 else np.nan,
        'OOS_EMSE': float(emse_num / max(emse_den,1))
    }
    return y_hat, metrics

#deciles
def decile_sorts_safe(meta_df, pred, ret_col='r_dh_no_tc', w_col='doi', time_col='month'):
    df = meta_df[[time_col, ret_col, w_col]].copy()
    df['pred'] = np.asarray(pred, float)
    df = df.dropna(subset=['pred', ret_col])
    #ensure month dtype
    df[time_col] = pd.to_datetime(df[time_col], errors='coerce').dt.to_period('M').dt.to_timestamp()
    out = []
    for mth, g in df.groupby(time_col, observed=True):
        g = g.sort_values('pred')
        #need at least 10 obs and at least 2 unique preds
        if len(g) < 10 or g['pred'].nunique() < 2:
            continue
        g['dec'] = pd.qcut(g['pred'], 10, labels=False, duplicates='drop') + 1
        ew = g.groupby('dec')[ret_col].mean()
        w = pd.to_numeric(g[w_col], errors='coerce').clip(lower=0).fillna(0.0)
        Wsum = w.groupby(g['dec']).sum().replace(0, np.nan)
        oiw = (g[ret_col]*w).groupby(g['dec']).sum() / Wsum
        if 1 in ew.index and 10 in ew.index:
            out.append({'month': mth,
                        'EW_LS': float(ew.loc[10]-ew.loc[1]),
                        'OIW_LS': float(oiw.loc[10]-oiw.loc[1])})
    if not out:
        return pd.DataFrame(columns=['month','EW_LS','OIW_LS']), {
            'months': 0, 'EW_LS_mean': np.nan, 'EW_LS_ann_sharpe': np.nan,
            'OIW_LS_mean': np.nan, 'OIW_LS_ann_sharpe': np.nan
        }
    res = pd.DataFrame(out).sort_values('month')
    summ = {
        'months': int(len(res)),
        'EW_LS_mean': float(res['EW_LS'].mean()),
        'EW_LS_ann_sharpe': float(np.sqrt(12)*res['EW_LS'].mean()/ (res['EW_LS'].std(ddof=1)+1e-12)) if len(res)>1 else np.nan,
        'OIW_LS_mean': float(res['OIW_LS'].mean()),
        'OIW_LS_ann_sharpe': float(np.sqrt(12)*res['OIW_LS'].mean()/ (res['OIW_LS'].std(ddof=1)+1e-12)) if len(res)>1 else np.nan,
    }
    return res, summ


meta_fix = ensure_month_doi(meta)
splits = load_splits(res=res) 

meta_pca, pca_models, pinfo = compute_pcs_expanding(Xz, meta_fix, splits, var_threshold=0.99, k_max=8)
print(f"[PCA] K={pinfo['K']} | cumvar(first train)={pinfo['cumvar_first_train']:.3f}")

y_hat_pc, metrics = run_pca_baseline(meta_pca, splits, target_col='r_dh_no_tc', weight_col='doi')
print("[Baseline] OOS WMSE:", metrics['OOS_WMSE'], "| OOS EW-MSE:", metrics['OOS_EMSE'])

deciles_ts, deciles_sum = decile_sorts_safe(meta_pca, y_hat_pc, ret_col='r_dh_no_tc', w_col='doi', time_col='month')
print("[Deciles] summary:", deciles_sum)


In [ ]:
import matplotlib.pyplot as plt

#Newey–West t-stat
def newey_west_t(x, L=6):
    x = pd.to_numeric(pd.Series(x).dropna(), errors='coerce').dropna().to_numpy(float)
    n = len(x)
    if n == 0:
        return np.nan, np.nan, np.nan, 0
    m = x.mean(); u = x - m
    def acov(l): return np.dot(u[l:], u[:-l]) / n if l>0 else np.dot(u, u) / n
    S = acov(0)
    for l in range(1, min(L, n-1)+1):
        w = 1.0 - l/(L+1.0)
        S += 2.0 * w * acov(l)
    se = np.sqrt(S / n); t = m / se if se>0 else np.nan
    return float(m), float(se), float(t), n

#Rank-IC (Spearman)
def rank_ic_series(meta_df, pred, ret_col='r_dh_no_tc', time_col='month'):
    df = meta_df[[time_col, ret_col]].copy()
    df['pred'] = np.asarray(pred, float)
    df = df.dropna(subset=[time_col, ret_col, 'pred'])
    df[time_col] = pd.to_datetime(df[time_col], errors='coerce').dt.to_period('M').dt.to_timestamp()
    out = []
    for m, g in df.groupby(time_col, observed=True):
        if len(g) < 3: 
            continue
        rp = g['pred'].rank(method='average'); rr = g[ret_col].rank(method='average')
        ic = np.corrcoef(rp, rr)[0,1]
        if np.isfinite(ic): out.append({'month': m, 'IC': float(ic)})
    ic_df = pd.DataFrame(out).sort_values('month')
    mu, se, t, n = newey_west_t(ic_df['IC'], L=6) if len(ic_df) else (np.nan,)*4
    return ic_df, {'mean_IC':mu,'NW_se':se,'NW_t':t,'n_months':n}

#deciles long sort
def decile_ts(meta_df, pred, ret_col='r_dh_no_tc', w_col='doi', time_col='month'):
    df = meta_df[[time_col, ret_col, w_col]].copy()
    df['pred'] = np.asarray(pred, float)
    df = df.dropna(subset=[time_col, ret_col, 'pred'])
    df[time_col] = pd.to_datetime(df[time_col], errors='coerce').dt.to_period('M').dt.to_timestamp()
    rows = []
    for m, g in df.groupby(time_col, observed=True):
        g = g.sort_values('pred')
        if len(g) < 10 or g['pred'].nunique() < 2: 
            continue
        g['dec'] = pd.qcut(g['pred'], 10, labels=False, duplicates='drop') + 1
        ew = g.groupby('dec')[ret_col].mean()
        w  = pd.to_numeric(g[w_col], errors='coerce').clip(lower=0).fillna(0.0)
        Wsum = w.groupby(g['dec']).sum().replace(0, np.nan)
        oiw = (g[ret_col]*w).groupby(g['dec']).sum() / Wsum
        if 1 in ew.index and 10 in ew.index:
            rows.append({'month': m, 'EW_LS': float(ew.loc[10]-ew.loc[1]),
                                  'OIW_LS': float(oiw.loc[10]-oiw.loc[1])})
    ts = pd.DataFrame(rows).sort_values('month')
    ew_mu, ew_se, ew_t, _   = newey_west_t(ts['EW_LS'],  L=6) if len(ts) else (np.nan,)*4
    oiw_mu, oiw_se, oiw_t, _= newey_west_t(ts['OIW_LS'], L=6) if len(ts) else (np.nan,)*4
    return ts, {'months':int(len(ts)),
                'EW_LS_mean':ew_mu, 'EW_LS_seNW':ew_se, 'EW_LS_tNW':ew_t,
                'OIW_LS_mean':oiw_mu,'OIW_LS_seNW':oiw_se,'OIW_LS_tNW':oiw_t}

def decile_bars(meta_df, pred, ret_col='r_dh_no_tc', w_col='doi'):
    df = meta_df[[ret_col, w_col]].copy()
    df['pred'] = np.asarray(pred, float)
    df = df.dropna(subset=['pred', ret_col])
    if df['pred'].nunique() < 10:
        return None, None
    df['dec'] = pd.qcut(df['pred'], 10, labels=False, duplicates='drop') + 1
    ew  = df.groupby('dec')[ret_col].mean()
    w   = pd.to_numeric(df[w_col], errors='coerce').clip(lower=0).fillna(0.0)
    Wsum= w.groupby(df['dec']).sum().replace(0, np.nan)
    oiw = (df[ret_col]*w).groupby(df['dec']).sum() / Wsum
    return ew, oiw

#pca explained variance
def pca_explained_variance_curve(Xz, splits, k_max=20):
    N = Xz.shape[0]; X = Xz.reshape(N, -1).astype(np.float32)
    first_y = sorted(splits.keys())[0]; tr0 = splits[first_y]['train_idx']
    try:
        from sklearn.decomposition import PCA as SkPCA
        evr = SkPCA(n_components=min(k_max, X.shape[1]), svd_solver='full', random_state=0)\
                .fit(X[tr0]).explained_variance_ratio_
    except Exception:
        Xc = X[tr0] - X[tr0].mean(axis=0); _, S, _ = np.linalg.svd(Xc, full_matrices=False)
        evr = (S**2)/(S**2).sum(); evr = evr[:k_max]
    cum = np.cumsum(evr)
    return evr, cum

#plots
def show_rank_ic_ts(ic_df):
    if ic_df is None or len(ic_df)==0: 
        print("No IC data."); return
    plt.figure()
    s = ic_df.set_index('month')['IC']
    plt.plot(s.index, s.values)
    plt.axhline(0, linewidth=1)
    plt.title("Monthly Rank-IC (Spearman) — PCA baseline")
    plt.ylabel("IC"); plt.xlabel("Month")
    plt.tight_layout(); plt.show()

def show_ls_ts(ls_ts):
    if ls_ts is None or len(ls_ts)==0:
        print("No LS time-series."); return
    # EW
    plt.figure()
    s = ls_ts.set_index('month')['EW_LS']
    plt.plot(s.index, s.values); plt.axhline(0, linewidth=1)
    plt.title("D10–D1 (EW)"); plt.ylabel("Return"); plt.xlabel("Month")
    plt.tight_layout(); plt.show()
    # OIW
    plt.figure()
    s = ls_ts.set_index('month')['OIW_LS']
    plt.plot(s.index, s.values); plt.axhline(0, linewidth=1)
    plt.title("D10–D1 (OIW)"); plt.ylabel("Return"); plt.xlabel("Month")
    plt.tight_layout(); plt.show()

def show_decile_bars(ew, oiw):
    if ew is None:
        print("Not enough unique predictions to form deciles."); return
    d = ew.index.astype(int)
    plt.figure()
    plt.bar(d-0.15, ew.values, width=0.3, label="EW")
    if oiw is not None:
        plt.bar(d+0.15, oiw.values, width=0.3, label="OIW")
    plt.xlabel("Decile (by prediction)"); plt.ylabel("Mean return"); plt.title("Average return by decile")
    plt.legend(); plt.tight_layout(); plt.show()

def show_evr(evr, cum):
    ks = np.arange(1, len(evr)+1)
    plt.figure()
    plt.bar(ks, evr)
    plt.plot(ks, cum, marker='o', linewidth=1.5)
    plt.xlabel("Principal component"); plt.ylabel("Share"); plt.title("PCA explained variance (first OOS train)")
    plt.tight_layout(); plt.show()

def show_pc_heatmaps(pca_models, splits, grid_days, grid_deltas, K_plot=3):
    first_y = sorted(pca_models.keys())[0]
    comps = pca_models[first_y].components_
    H, W = len(grid_days), len(grid_deltas)
    Kp = min(K_plot, comps.shape[0])
    for j in range(Kp):
        M = comps[j].reshape(H, W)
        plt.figure()
        im = plt.imshow(M, aspect='auto', origin='lower')
        plt.colorbar(im, fraction=0.046, pad=0.04)
        plt.yticks(ticks=np.arange(H), labels=[str(d) for d in grid_days])
        plt.xticks(ticks=np.arange(W), labels=[str(d) for d in grid_deltas], rotation=90)
        plt.xlabel("Delta bucket"); plt.ylabel("Days to maturity")
        plt.title(f"PC{j+1} loadings (first OOS year = {first_y})")
        plt.tight_layout(); plt.show()


ic_df, ic_summ = rank_ic_series(meta_pca, y_hat_pc, ret_col='r_dh_no_tc', time_col='month')
print("[Rank-IC] mean_IC={:.4f}, NW_se={:.4f}, NW_t={:.2f}, months={}".format(
    ic_summ['mean_IC'], ic_summ['NW_se'], ic_summ['NW_t'], ic_summ['n_months']))
show_rank_ic_ts(ic_df)

ls_ts, ls_summ = decile_ts(meta_pca, y_hat_pc, ret_col='r_dh_no_tc', w_col='doi', time_col='month')
print("[Deciles LS] months={months}, EW mean={EW_LS_mean:.4f} (t_NW={EW_LS_tNW:.2f}), "
      "OIW mean={OIW_LS_mean:.4f} (t_NW={OIW_LS_tNW:.2f})".format(**ls_summ))
show_ls_ts(ls_ts)

ew_bar, oiw_bar = decile_bars(meta_pca, y_hat_pc, ret_col='r_dh_no_tc', w_col='doi')
show_decile_bars(ew_bar, oiw_bar)

evr, cum = pca_explained_variance_curve(Xz, splits, k_max=12)
show_evr(evr, cum)

show_pc_heatmaps(pca_models, splits, res['grid_days'], res['grid_deltas'], K_plot=3)
